In [2]:
## Importing the required python libraries 
import os, sys
import warnings
import timeit
import numpy as np
import pandas as pd 
import pkg_resources
import matplotlib.pyplot as plt
import re
import seaborn as sns
from collections import defaultdict


In [3]:
## Importing pergene files 

pergene_files=[]
#data_dir= "../satay/data_files/data_unmerged/"
#data_dir="../transposonmapper/data_files/files4test/"
data_dir="../postprocessed-data/"
#data_dir="../transposonmapper/data_files/"
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith("pergene_insertions.xlsx"):
            pergene_files.append(os.path.join(root, file))

list_data=[]
for i in pergene_files:
    list_data.append(pd.read_excel(i,engine='openpyxl',index_col="Unnamed: 0"))

keys=[]
for i in np.arange(0,len(pergene_files)):
    keys.append(pergene_files[i].split("/")[-1].split("_")[0]+"_"+pergene_files[i].split("/")[-1].split("_")[1])

list_data_pd=pd.concat(list_data,axis=0,keys=keys)

In [4]:
## Importing postproccesed data from pipeline

data_post=[]

for i in keys:
    data_post.append(pd.read_excel(data_dir+i+".xlsx",index_col="Unnamed: 0",engine='openpyxl'))
    
list_data_post=pd.concat(data_post,axis=0,keys=keys)
list_data_post.drop(columns=["Feature_alias","Feature_name"],inplace=True)
list_data_post.fillna(1,inplace=True)


In [4]:
list_data_post.head(2)
# list_data_pd.head()

Standard_name Feature_type  Essentiality Chromosome  Position  \
dbem3_b 0     noncoding            1           1.0          I    [0, 1]   
        1        TEL01L     Telomere           1.0          I  [2, 335]   

           Nbasepairs  Ninsertions  Ninsertions_truncatedgene  Nreads  \
dbem3_b 0           2            0                          0       0   
        1         334           38                         38    2356   

                                                 Nreads_list  \
dbem3_b 0                                               [0.]   
        1  [  0.   0.   0.   0.   0.   0.   0.   0.   0. ...   

           Nreads_truncatedgene  Nreadsperinsrt  Nreadsperinsrt_truncatedgene  
dbem3_b 0                     0        0.000000                           0.0  
        1                  2356       63.675676                          62.0

In [5]:
## Importing gene protein length , and protein domains coordinates of every gene 

protein_length=pd.read_csv('../data/Gene_Protein_length_yeastmine.tsv',sep="\t")
protein_length.index=protein_length.loc[:,"Gene Name"]
protein_length.drop(columns="Gene Name",inplace=True)

domains_coordinates=pd.read_csv('../data/Domains_all_genes_protein_coordinates_yeastmine.tsv',sep="\t")

domains_coordinates.dropna(inplace=True)
gene_with_introns=pd.read_csv('../data/Gene_Introns.tsv',sep="\t")



In [6]:
## Combine protein length and protein domains coordinates
domains_coordinates.loc[:,"Protein Length"]=np.nan
for gene_name in protein_length.index: 
    tmp=np.where(domains_coordinates.loc[:,"Gene Name"]==gene_name)[0]
    if tmp.size!=0: 
        for i in np.arange(0,len(tmp)):
            domains_coordinates.loc[tmp[i], "Protein Length"]=protein_length.loc[gene_name, "Protein Length"]

domains_coordinates.dropna(inplace=True) # to drop the proteins that do not have protein domains annotated
domains_coordinates.index=domains_coordinates.loc[:,"Gene Name"]
domains_coordinates.drop(columns="Gene Name",inplace=True)

In [7]:
domains_coordinates.head(2)

,Protein Domain,Protein domain description,Protein start location,Protein End location,Method,Domain ID,Protein Length
Gene Name,,,,,,,
AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,10,106,Pfam,2106639,309.0
AAC1,PF00153,Mitochondrial carrier protein; Mitochondrial s...,116,210,Pfam,2106640,309.0


In [7]:
## Add for every gene name from the pergene files , a column with an array of the protein domains
# founded in the domains coordinates file, as well as the length of the protein,and the
# end location and start location of every protein domain
list_data_domains_data=list_data_pd.copy()
df=list_data_domains_data.loc[keys[0]]
df.set_index("Gene name",inplace=True)

    


In [8]:

## make a dictionary with protein domains locations per gene , the solution of an array and attaching it
# a dataframe do not work, so I will use a dictionary
domain_locations=defaultdict(dict)

for j in df.index:
    if j in domains_coordinates.index:
        if len([domains_coordinates.loc[j,"Protein Domain"]])>1:
            for i in np.arange(0,len(domains_coordinates.loc[j,"Protein Domain"].tolist())):
            
                domain_locations[j][i]=(np.array([domains_coordinates.loc[j,"Protein start location"][i],domains_coordinates.loc[j,
        "Protein End location"][i]]))
        else:
            domain_locations[j]=(np.array([domains_coordinates.loc[j,"Protein start location"],domains_coordinates.loc[j,
    "Protein End location"]]))
    else:
        domain_locations[j]=np.nan


domains_names=defaultdict(dict)


for j in df.index:
    if j in domains_coordinates.index:
        if type(domains_coordinates.loc[j,"Protein Domain"])!=str:
            if len(domains_coordinates.loc[j,"Protein Domain"].tolist())>1:
                for i in np.arange(0,len(domains_coordinates.loc[j,"Protein Domain"].tolist())):
                    domains_names[j][i]=np.array(domains_coordinates.loc[j,"Protein Domain"][i])
            else:
                domains_names[j]=np.array(domains_coordinates.loc[j,"Protein Domain"])
        else:
            domains_names[j]=np.array(domains_coordinates.loc[j,"Protein Domain"])
    else:
        domains_names[j]=np.nan
        


for i in df.index:
    if i in domains_coordinates.index:
        df.loc[i,"protein length"]=np.unique(domains_coordinates.loc[i,"Protein Length"]).tolist()

    else:
        df.loc[i,"protein length"]=np.nan



/data/localhome/linigodelacruz/anaconda3/envs/transposonmapper/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/data/localhome/linigodelacruz/anaconda3/envs/transposonmapper/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/data/localhome/linigodelacruz/anaconda3/envs/transposonmapper/lib/python3.9/site-packages/pan

In [9]:
domains_names_df=pd.DataFrame.from_dict(domains_names,orient="index")
domains_names_df.columns=["protein domain"]

domain_locations_df=pd.DataFrame.from_dict(domain_locations,orient="index")
domain_locations_df.columns=["protein locations"]


In [10]:
df_merged=pd.merge(df,domain_locations_df,left_index=True,right_index=True)

df_merged_II=pd.merge(df_merged,domains_names_df,left_index=True,right_index=True)

df_merged_II.loc["AAC1"]

Chromosome                                                                   XIII
Start location                                                             387315
End location                                                               388244
Insertion locations             [387338, 387376, 387380, 387386, 387393, 38740...
Reads per insertion location    [9, 89, 2, 2, 1, 2, 4, 4, 2, 1, 1, 5, 3, 1, 1,...
Reads                                                                        2563
Insertions                                                                     77
protein length                                                              309.0
protein locations                               [[10, 116, 217], [106, 210, 304]]
protein domain                               {0: PF00153, 1: PF00153, 2: PF00153}
Name: AAC1, dtype: object

In [253]:
## Example on how to translate protein domains coordinates to genomic coordinates
from from_excel_to_list import from_excel_to_list
gene="AAC1"

genomic_length=df_merged_II.loc[gene,"End location"]-df_merged_II.loc[gene,"Start location"]
protein_length=df_merged_II.loc[gene,"protein length"]

domain_1_protein=[df_merged_II.loc[gene,"protein locations"][1][0],df_merged_II.loc[gene,"protein locations"][0][0]]

domain_1_protein_rel=domain_1_protein/protein_length

domain_1_genomic_rel=np.round(domain_1_protein_rel*genomic_length)

domain_1_genomic=[domain_1_genomic_rel[1]+df_merged_II.loc[gene,"Start location"],
domain_1_genomic_rel[0]+df_merged_II.loc[gene,"Start location"]]

insertions_vector=from_excel_to_list(df_merged_II.loc[gene,"Insertion locations"])
reads_vector=from_excel_to_list(df_merged_II.loc[gene,"Reads per insertion location"])

pos_domain_1=[]

for i in insertions_vector:
    if domain_1_genomic[0]<=i and i<=domain_1_genomic[1]:
        j=np.where(insertions_vector.index(i))[0][0]
        print(j)
        pos_domain_1.append(reads_vector[j])
       

reads_domain_1=np.sum(pos_domain_1)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [171]:
domain_1_genomic[0],domain_1_genomic[1],reads_domain_1/np.sum(reads_vector)

(387345.0, 387634.0, 0.050865418580007066)

In [250]:
a=[2,5]
a.index(2)

0

In [296]:
from from_excel_to_list import from_excel_to_list
domain_genomic=defaultdict(dict)
reads_genomic=defaultdict(dict)

for gene in df_merged_II.index: 


    genomic_length=df_merged_II.loc[gene,"End location"]-df_merged_II.loc[gene,"Start location"]
    protein_length=df_merged_II.loc[gene,"protein length"]

    if type(df_merged_II.loc[gene,"protein locations"])!=float:
        
         tmp=df_merged_II.loc[gene,"protein locations"].tolist()

         if type(tmp[0])!=int:

            for i in np.arange(0,len(tmp[0])): 
                
            
                domain_protein=[df_merged_II.loc[gene,"protein locations"][1][i],
            df_merged_II.loc[gene,"protein locations"][0][i]] # end location first

                domain_protein_rel=domain_protein/protein_length

                domain_genomic_rel=np.round(domain_protein_rel*genomic_length)

                domain_genomic[gene][i]=([domain_genomic_rel[1]+df_merged_II.loc[gene,"Start location"],
            domain_genomic_rel[0]+df_merged_II.loc[gene,"Start location"]])
            
         else: 
             
                
            domain_protein=[df_merged_II.loc[gene,"protein locations"][1],
            df_merged_II.loc[gene,"protein locations"][0]]

            domain_protein_rel=domain_protein/protein_length

            domain_genomic_rel=np.round(domain_protein_rel*genomic_length)

            domain_genomic[gene][0]=([domain_genomic_rel[1]+df_merged_II.loc[gene,"Start location"],
            domain_genomic_rel[0]+df_merged_II.loc[gene,"Start location"]])

                

                
            


In [1]:
for gene in df_merged_II.index:

    insertions_vector=from_excel_to_list(df_merged_II.loc[gene,"Insertion locations"])
    reads_vector=from_excel_to_list(df_merged_II.loc[gene,"Reads per insertion location"]) 

    if type(insertions_vector)!=int:

    

        for i in np.arange(0,len(insertions_vector)):
           
            pos_domain=[]
            if len(domain_genomic[gene])!=0:
                
                for k in np.arange(0,len(domain_genomic[gene])):

                    if domain_genomic[gene][k][0]<=insertions_vector[i] and insertions_vector[i]<=domain_genomic[gene][k][1]:
                        
                        pos_domain.append(reads_vector[i])
                        print(pos_domain)
                

                    reads_genomic[gene][k]=np.sum(pos_domain)
            else:
                reads_genomic[gene]=0
                   

NameError: name 'df_merged_II' is not defined

In [279]:

df_merged_II.loc["PAU8"]

Chromosome                                                                      I
Start location                                                               1807
End location                                                                 2169
Insertion locations             [1810, 1819, 1826, 1831, 1833, 1838, 1846, 188...
Reads per insertion location    [3, 30, 16, 2, 8, 1, 12, 50, 1, 3, 2, 1, 44, 2...
Reads                                                                         416
Insertions                                                                     29
protein length                                                              120.0
protein locations                                                       [22, 114]
protein domain                                                            PF00660
Name: PAU8, dtype: object

(array([], dtype=int64),)

In [67]:
domains_coordinates.index

Index(['AAC1', 'AAC1', 'AAC1', 'AAC3', 'AAC3', 'AAC3', 'AAD10', 'AAD14',
       'AAD3', 'AAD4',
       ...
       'ZRC1', 'ZRT1', 'ZRT2', 'ZRT3', 'ZTA1', 'ZTA1', 'ZUO1', 'ZUO1', 'ZWF1',
       'ZWF1'],
      dtype='object', name='Gene Name', length=7327)

In [52]:
df.loc["AAC1","Protein domains locations"]=b[0][0]

In [41]:
np.concatenate(b,axis=0).tolist()

[10, 106, 116, 210, 217, 304]

In [ ]:
df.loc[tmp,"Gene name"]

'AAC1'